# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298485256655                   -0.85    4.5
  2   -8.300200911860       -2.77       -1.25    1.5
  3   -8.300447224397       -3.61       -1.87    3.5
  4   -8.300462245067       -4.82       -2.73    2.1
  5   -8.300464318658       -5.68       -3.15    3.6
  6   -8.300464474066       -6.81       -3.29    8.5
  7   -8.300464556430       -7.08       -3.43    1.4
  8   -8.300464603254       -7.33       -3.57    1.9
  9   -8.300464633570       -7.52       -3.76    1.6
 10   -8.300464640173       -8.18       -3.91    6.6
 11   -8.300464643878       -8.43       -4.41    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67450161026                   -0.70    5.8
  2   -16.67867100621       -2.38       -1.14    1.0
  3   -16.67922341265       -3.26       -1.86    3.8
  4   -16.67927930110       -4.25       -2.75    3.4
  5   -16.67928595510       -5.18       -3.20    4.0
  6   -16.67928621187       -6.59       -3.50    5.0
  7   -16.67928622070       -8.05       -3.93    1.0
  8   -16.67928622401       -8.48       -4.61    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32535665029                   -0.56    6.4
  2   -33.33263075605       -2.14       -1.00    1.2
  3   -33.33591369896       -2.48       -1.69    4.1
  4   -33.33610780705       -3.71       -2.56    2.9
  5   -33.33684524659       -3.13       -2.32    7.9
  6   -33.33685789837       -4.90       -2.35    1.0
  7   -33.33694369314       -4.07       -3.44    5.6
  8   -33.33694344522   +   -6.61       -3.44    6.2
  9   -33.33694391130       -6.33       -4.03    6.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298561372242                   -0.85    4.5
  2   -8.300278402291       -2.77       -1.59    1.0
  3   -8.300456760567       -3.75       -2.61    7.6
  4   -8.300412611663   +   -4.36       -2.40   12.0
  5   -8.300464389499       -4.29       -3.57    3.1
  6   -8.300464584290       -6.71       -3.92    3.2
  7   -8.300464643253       -7.23       -4.83    4.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32428259262                   -0.56    6.6
  2   -33.30355848987   +   -1.68       -1.26    1.2
  3   -3.517328531905   +    1.47       -0.33    8.9
  4   -33.32704022007        1.47       -1.75    8.1
  5   -33.22894819460   +   -1.01       -1.29    3.9
  6   -32.69541999851   +   -0.27       -1.16    5.9
  7   -33.29141435272       -0.22       -1.70    4.6
  8   -33.33453796159       -1.37       -2.36    3.4
  9   -33.33670820908       -2.66       -2.76    2.8
 10   -33.33674639553       -4.42       -2.87    3.2
 11   -33.33689291579       -3.83       -2.92    2.8
 12   -33.33692215978       -4.53       -3.27    2.0
 13   -33.33694137563       -4.72       -3.74    2.9
 14   -33.33694325843       -5.73       -3.98    4.4
 15   -33.33694377129       -6.29       -4.27    4.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.